In [13]:
%pip install Tweepy

     |████████████████████████████████| 153kB 2.9MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [92]:
import tweepy
import json
import pandas as pd
import time
#simport Manage_Files


ModuleNotFoundError: No module named 'Manage_Files'

In [11]:
#Variable de autentificación en Twitter
OAUTH_TOKEN='1204477616398503936-7Z5WSIa0490S7HbIoPrDzosdA8aJKb'
OAUTH_SECRET='OCHWxZGQlrnaoNx4corTWHNZPrtGtItteYnow7nMDhtON'
CONSUMER_KEY='IRuah0diiQDx7ayuHaFmCOvTq'
CONSUMER_SECRET='FkYZE9vZXWMgWvovRjcqQaixPleh6ADXdD3XU05ZsIkscSFzXd'

In [12]:
auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN,OAUTH_SECRET)
api = tweepy.API(auth)

In [16]:
#Búsqueda de Tweets por palabra
def busqueda_tweets_palabra(palabra="Ryanair", lenguaje="es", numero_de_Tweets=200):
    return tweepy.Cursor(api.search, palabra, lang=lenguaje).items(numero_de_Tweets)

tweets=busqueda_tweets_palabra()
outtweets = [(tweet.created_at, 
              tweet.text, 
              tweet.retweet_count, 
              str(tweet.favorite_count)+'') for tweet in tweets]
outtweets

[(datetime.datetime(2019, 12, 18, 19, 3, 53),
  'Ryanair atribuye los recortes en Girona a los problemas del Boeing 737 MAX https://t.co/asx2LZmyV3 vía @elpais_economia',
  0,
  '0'),
 (datetime.datetime(2019, 12, 18, 19, 0, 8),
  'Los asientos separados, el nuevo mercadillo de las low cost (Ryanair, Vueling, etc...) https://t.co/Zg75Rhm4re',
  0,
  '0'),
 (datetime.datetime(2019, 12, 18, 18, 49, 18),
  'RT @marcosleyley: He visto sardinas en lata que tienen más espacio que volar en Vueling o Ryanair. Es curioso que mientras la población mun…',
  3,
  '0'),
 (datetime.datetime(2019, 12, 18, 18, 48, 17),
  'si tienes cualquier problema para que puedas resolverlo "perfectamente" te ponen un número de atención al cliente d… https://t.co/e94lP2nyZ5',
  0,
  '1'),
 (datetime.datetime(2019, 12, 18, 18, 40),
  'viajar con @Ryanair_ES y otras maneras de solo tener problemas.',
  0,
  '2'),
 (datetime.datetime(2019, 12, 18, 18, 36, 2),
  'Inspección de Trabajo abrirá expediente ✈️ https://t.co/

In [4]:
#El getter de Twitter contiene limitación al obtener los datos ya que devuelve 5000 cuando en realidad tiene 285055
def obtener_follower_ids():
    return api.followers_ids('@elcorteingles')

print(len(obtener_follower_ids()))
      

5000


In [104]:
#Nueva forma de obtener todas las cuentas que siguen a El Corte Inglés/Cabify
#cabify_espana
#elcorteingles
#NokiaMobile
# INFORMACION DE LIMITACION: https://developer.twitter.com/en/docs/basics/rate-limits
def get_followers_ids(api, twitter_account='@NokiaMobile'):
    for page in tweepy.Cursor(api.followers_ids, id=twitter_account, count=5000).pages():
        followers_ids.extend(page)
        time.sleep(60*15)

   # except tweepy.RateLimitError:
        print("RateLimitError...Esperamos para poder continuar por la limitación de la API")
    #    time.sleep(60*15)
        #for page in tweepy.Cursor(api.followers_ids, id=twitter_account, count=5000).pages():
        #    followers_ids.extend(page)
    
    return followers_ids

followers_ids=[]
get_followers_ids(api)
print(len(followers_ids))


RateLimitError...Esperamos para poder continuar por la limitación de la API
RateLimitError...Esperamos para poder continuar por la limitación de la API


TweepError: Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /1.1/followers/ids.json?cursor=1650696684791298305&id=%40NokiaMobile&count=5000 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x11e9004d0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [98]:
#Guardado de los ids en los ficheros para agilizar el desarrollo
dataFollowers_id= pd.DataFrame(followers_ids)

In [29]:
#Nueva forma de obtener todas las cuentas sigue El Corte Inglés/Cabify
#cabify_espana
#elcorteingles

# INFORMACION DE LIMITACION: https://developer.twitter.com/en/docs/basics/rate-limits
def get_following_ids(api, twitter_account='@cabify_espana'):
    try:
        for page in tweepy.Cursor(api.friends_ids, id=twitter_account, count=5000).pages():
            following_ids.extend(page)

    except tweepy.RateLimitError:
        print("RateLimitError...Esperamos para poder continuar por la limitación de la API")
        time.sleep(60*15)
        for page in tweepy.Cursor(api.friends_ids, id=twitter_account, count=5000).pages():
            following_ids.extend(page)
    return following_ids

following_ids=[]
get_following_ids(api)
print(len(following_ids))


630


In [99]:
#Guardado de los ids en los ficheros para agilizar el desarrollo
dataFollowing_id= pd.DataFrame(following_ids)


In [23]:
#cabify_espana
#elcorteingles
def obtener_friends_ids():
    return api.friends_ids('@cabify_espana')

print(len(obtener_friends_ids()))

630


In [78]:
#Obtención de las cuentas que siguen El Corte Inglés
def get_info_followers(limit=5000):
    cursor = -1
    followers_list=list()
    while cursor != 0:
        ids_follow,cursor_followers = api.followers_ids('@elcorteingles', cursor=cursor, count=limit)
        followers_list = followers_list.append(ids_follow)
        cursor = cursor_followers
        time.sleep(60)
    return followers_list

lista_followers= get_info_followers()

AttributeError: 'NoneType' object has no attribute 'append'

In [48]:
#Obtener información de un IDs
info=api.lookup_users(user_ids=[1212959365398839296])
print(info)

[User(_api=<tweepy.api.API object at 0x10ded7e10>, _json={'id': 1212959365398839296, 'id_str': '1212959365398839296', 'name': 'Natalia Escubilla', 'screen_name': 'NatyEscubilla', 'location': 'Buenos Aires Argentina', 'description': '💞 Cantante 🎙️Todos nuestros sueños pueden hacerse realidad, si tenemos el coraje de perseguirlos.', 'url': 'https://t.co/7Bml7PpzqS', 'entities': {'url': {'urls': [{'url': 'https://t.co/7Bml7PpzqS', 'expanded_url': 'https://www.youtube.com/channel/UCMxu_CmZPkVAaBn03dlMXsw', 'display_url': 'youtube.com/channel/UCMxu_…', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 133, 'friends_count': 414, 'listed_count': 0, 'created_at': 'Fri Jan 03 04:50:35 +0000 2020', 'favourites_count': 2, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 8, 'lang': None, 'status': {'created_at': 'Sat Jan 04 17:20:05 +0000 2020', 'id': 1213510424500559875, 'id_str': '1213510424500559875', 'te

In [53]:
cota=[following_ids[:4]]
print(cota)

[[48422383, 1173984355800293377, 2992924523, 715535722086801408]]


In [90]:
# Existe limitación al traer la información de los seguidores
data = []
for follower in cota:
    follower_obj=api.lookup_users(user_ids=follower, include_entities=False) 
    for user in follower_obj:
        follow={}
        follow["id"]=user.id
        follow["name"]=user.name
        follow["created_at"]=user.created_at
        follow["followers_count"]=user.followers_count
        data.append(follow)

dataframe = pd.DataFrame(data)
dataframe

,id,name,created_at,followers_count
0,48422383,Jonathan Chacón,2009-06-18 17:29:37,2466
1,1173984355800293377,Mobilidade Coruñesa,2019-09-17 15:37:47,1
2,2992924523,Cabify,2015-01-22 20:30:57,2149
3,715535722086801408,Mutua Madrileña,2016-03-31 13:46:24,4814


In [ ]:
#Guardado en ficheros de los IDS de Twitter
def saveID(dataframe,file):
    dataframe.to_csv(file)

saveID(dataFollowing_id,'followings_id.csv')
saveID(dataFollowers_id,'followers_id.csv')